This notebook compares analytical and numerical evaluations of alchemical derivatives of a rotated and translated CH3OH molecule up to the third order in Hartree-Fock theory.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
from scipy.spatial.transform import Rotation

pyscf.__version__

'2.2.1'

In [2]:
from FcMole import FcM_like

Rotating and translating CH3OH.

In [3]:
mol_coord = np.array([[-1.27549516,	0.03357710,	0.00000000],
                      [-2.10170657,	-0.87840808,	1.59107621],
                      [-2.10170657,	-0.87840808,	-1.59107621],
                      [-1.76544024,	1.99528228,	0.00000000],
                      [1.41253482,	-0.22423026,	0.00000000],
                      [2.19050133,	1.29874374,	0.00000000]])
# Coordinates in Angstrom
# C	-0.67496302	0.01776824	0.00000000
# H	-1.11217530	-0.46483357	0.84196133
# H	-1.11217530	-0.46483357	-0.84196133
# H	-0.93423081	1.05585799	0.00000000
# O	0.74748129	-0.11865755	0.00000000
# H	1.15916347	0.68726564	0.00000000
rotvec = np.array([1.2 * np.pi, 1.8 * np.pi, 0.8 * np.pi])
rot = Rotation.from_rotvec(rotvec)
mol_coord = rot.apply(mol_coord)
# print(mol_coord)
# print(np.linalg.norm(mol_coord[1]))
for i in range(6):
  mol_coord[i] += [22.4, 8.7, 11.8]
print(mol_coord)

[[21.51976504  8.14268562 12.53661552]
 [22.19523369  6.50972431 13.49736794]
 [19.9104428   7.49931626 11.51581996]
 [20.96614982  9.55062923 13.87808426]
 [23.39549112  9.16151172 10.8826461 ]
 [23.76170114 10.78607451 11.27170604]]


In [4]:
# target_mol = """
# C	-1.27549516	0.03357710	0.00000000
# H	-2.10170657	-0.87840808	1.59107621
# H	-2.10170657	-0.87840808	-1.59107621
# H	-1.76544024	1.99528228	0.00000000
# O	1.41253482	-0.22423026	0.00000000
# H	2.19050133	1.29874374	0.00000000
# """
target_mol = []
atom_symbols = ['C', 'H', 'H', 'H', 'O', 'H']
for i, atom_coords in enumerate(mol_coord):
    # print(atom_coords)
    atom_str = f' {atom_symbols[i]} {atom_coords[0]: .8f} {atom_coords[1]: .8f} {atom_coords[2]: .8f}'
    print(atom_str)
    target_mol.append(atom_str)
print(target_mol)
# dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

 C  21.51976504  8.14268562  12.53661552
 H  22.19523369  6.50972431  13.49736794
 H  19.91044280  7.49931626  11.51581996
 H  20.96614982  9.55062923  13.87808426
 O  23.39549112  9.16151172  10.88264610
 H  23.76170114  10.78607451  11.27170604
[' C  21.51976504  8.14268562  12.53661552', ' H  22.19523369  6.50972431  13.49736794', ' H  19.91044280  7.49931626  11.51581996', ' H  20.96614982  9.55062923  13.87808426', ' O  23.39549112  9.16151172  10.88264610', ' H  23.76170114  10.78607451  11.27170604']


### Analytical evaluation

In [5]:
%load_ext autoreload
from AP_class import APDFT_perturbator as AP

In [6]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1,2,3,4,5],flag_response_property=True)

converged SCF energy = -115.087746889062


In [7]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force is not implemented in the Hartree-Fock theory

Inefficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [8]:
# inefficient_analytical_alchemical_force = np.zeros((6, 6, 3))
# for i in range(6):
#   inefficient_analytical_alchemical_force[i] = ap_nn.build_inefficient_alchemical_force(i)

Efficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [9]:
efficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  efficient_analytical_alchemical_force[i] = ap_nn.build_alchemical_force(i)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [10]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [11]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [12]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs2 = [-fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs3 = [0.0, fc_param, 0.0, 0.0, 0.0, 0.0]
fcs4 = [0.0, -fc_param, 0.0, 0.0, 0.0, 0.0]
fcs5 = [0.0, 0.0, fc_param, 0.0, 0.0, 0.0]
fcs6 = [0.0, 0.0, -fc_param, 0.0, 0.0, 0.0]
fcs7 = [0.0, 0.0, 0.0, fc_param, 0.0, 0.0]
fcs8 = [0.0, 0.0, 0.0, -fc_param, 0.0, 0.0]
fcs9 = [0.0, 0.0, 0.0, 0.0, fc_param, 0.0]
fcs10 = [0.0, 0.0, 0.0, 0.0, -fc_param, 0.0]
fcs11 = [0.0, 0.0, 0.0, 0.0, 0.0, fc_param]
fcs12 = [0.0, 0.0, 0.0, 0.0, 0.0, -fc_param]

##### Efficient numerical alchemical force

In [13]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
fmol5=FcM_like(mol_NN,fcs=fcs5)
fmol6=FcM_like(mol_NN,fcs=fcs6)
fmol7=FcM_like(mol_NN,fcs=fcs7)
fmol8=FcM_like(mol_NN,fcs=fcs8)
fmol9=FcM_like(mol_NN,fcs=fcs9)
fmol10=FcM_like(mol_NN,fcs=fcs10)
fmol11=FcM_like(mol_NN,fcs=fcs11)
fmol12=FcM_like(mol_NN,fcs=fcs12)
mf=scf.RHF(mol_NN)
mf1=scf.RHF(fmol1)
mf2=scf.RHF(fmol2)
mf3=scf.RHF(fmol3)
mf4=scf.RHF(fmol4)
mf5=scf.RHF(fmol5)
mf6=scf.RHF(fmol6)
mf7=scf.RHF(fmol7)
mf8=scf.RHF(fmol8)
mf9=scf.RHF(fmol9)
mf10=scf.RHF(fmol10)
mf11=scf.RHF(fmol11)
mf12=scf.RHF(fmol12)
# mf.xc = dft_functional
# mf1.xc = dft_functional
# mf2.xc = dft_functional
# mf3.xc = dft_functional
# mf4.xc = dft_functional
# mf5.xc = dft_functional
# mf6.xc = dft_functional
# mf7.xc = dft_functional
# mf8.xc = dft_functional
# mf9.xc = dft_functional
# mf10.xc = dft_functional
# mf11.xc = dft_functional
# mf12.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())
e5=mf5.scf(dm0=mf5.init_guess_by_1e())
e6=mf6.scf(dm0=mf6.init_guess_by_1e())
e7=mf7.scf(dm0=mf7.init_guess_by_1e())
e8=mf8.scf(dm0=mf8.init_guess_by_1e())
e9=mf9.scf(dm0=mf9.init_guess_by_1e())
e10=mf10.scf(dm0=mf10.init_guess_by_1e())
e11=mf11.scf(dm0=mf11.init_guess_by_1e())
e12=mf12.scf(dm0=mf12.init_guess_by_1e())

converged SCF energy = -115.08774688906
converged SCF energy = -115.102443949


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -115.07305281164
converged SCF energy = -115.088877458771
converged SCF energy = -115.086617404047
converged SCF energy = -115.088877458771
converged SCF energy = -115.086617404047
converged SCF energy = -115.088866074389
converged SCF energy = -115.086628787657
converged SCF energy = -115.110098300524
converged SCF energy = -115.065399080753
converged SCF energy = -115.088765014498
converged SCF energy = -115.086729720902


In [14]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()
nuc_grad5 = mf5.nuc_grad_method().kernel()
nuc_grad6 = mf6.nuc_grad_method().kernel()
nuc_grad7 = mf7.nuc_grad_method().kernel()
nuc_grad8 = mf8.nuc_grad_method().kernel()
nuc_grad9 = mf9.nuc_grad_method().kernel()
nuc_grad10 = mf10.nuc_grad_method().kernel()
nuc_grad11 = mf11.nuc_grad_method().kernel()
nuc_grad12 = mf12.nuc_grad_method().kernel()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0179489258    -0.0152830892     0.0130632867
1 H    -0.0130971946     0.0071837271    -0.0085023690
2 H     0.0112176449    -0.0033475533     0.0125853470
3 H     0.0056759678    -0.0036170712    -0.0083509582
4 O     0.0212575044     0.0572799579     0.0040951982
5 H    -0.0071049967    -0.0422159713    -0.0128905047
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0179275365    -0.0152769101     0.0130417101
1 H    -0.0130747539     0.0071167492    -0.0084613508
2 H     0.0111503588    -0.0033756686     0.0125485471
3 H     0.0056541683    -0.0035599259    -0.0082972841
4 O     0.0213010430     0.0573202136     0.0040651007
5 H    -0.0071032797    -0.0422244582    -0.0128967229
----------------------------------------------
--------------- RHF gradients ---------------
         x        

In [15]:
efficient_numerical_alchemical_force = np.zeros((6, 6, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)
efficient_numerical_alchemical_force[2] = (nuc_grad5 - nuc_grad6) / (2 * fc_param)
efficient_numerical_alchemical_force[3] = (nuc_grad7 - nuc_grad8) / (2 * fc_param)
efficient_numerical_alchemical_force[4] = (nuc_grad9 - nuc_grad10) / (2 * fc_param)
efficient_numerical_alchemical_force[5] = (nuc_grad11 - nuc_grad12) / (2 * fc_param)

#### Numerical alchemical derivatives

In [16]:
numerical_1st_alchemical_derivative = np.zeros((6))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)
numerical_1st_alchemical_derivative[2] = (e5 - e6) / (2 * fc_param)
numerical_1st_alchemical_derivative[3] = (e7 - e8) / (2 * fc_param)
numerical_1st_alchemical_derivative[4] = (e9 - e10) / (2 * fc_param)
numerical_1st_alchemical_derivative[5] = (e11 - e12) / (2 * fc_param)

#### Numerical alchemical hardness

In [17]:
ap_nn=AP(mf,sites=[0,1,2,3,4,5])
ap_nn1=AP(mf1,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn2=AP(mf2,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn3=AP(mf3,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn4=AP(mf4,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn5=AP(mf5,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn6=AP(mf6,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn7=AP(mf7,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn8=AP(mf8,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn9=AP(mf9,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn10=AP(mf10,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn11=AP(mf11,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn12=AP(mf12,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
an5 = ap_nn5.build_gradient()
an6 = ap_nn6.build_gradient()
an7 = ap_nn7.build_gradient()
an8 = ap_nn8.build_gradient()
an9 = ap_nn9.build_gradient()
an10 = ap_nn10.build_gradient()
an11 = ap_nn11.build_gradient()
an12 = ap_nn12.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
ann5 = ap_nn5.build_hessian()
ann6 = ap_nn6.build_hessian()
ann7 = ap_nn7.build_hessian()
ann8 = ap_nn8.build_hessian()
ann9 = ap_nn9.build_hessian()
ann10 = ap_nn10.build_hessian()
ann11 = ap_nn11.build_hessian()
ann12 = ap_nn12.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
dipole5 = ap_nn5.ref_elec_electric_dipole_moment * -1
dipole6 = ap_nn6.ref_elec_electric_dipole_moment * -1
dipole7 = ap_nn7.ref_elec_electric_dipole_moment * -1
dipole8 = ap_nn8.ref_elec_electric_dipole_moment * -1
dipole9 = ap_nn9.ref_elec_electric_dipole_moment * -1
dipole10 = ap_nn10.ref_elec_electric_dipole_moment * -1
dipole11 = ap_nn11.ref_elec_electric_dipole_moment * -1
dipole12 = ap_nn12.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1
pol5 = ap_nn5.ref_elec_electric_polarizability * -1
pol6 = ap_nn6.ref_elec_electric_polarizability * -1
pol7 = ap_nn7.ref_elec_electric_polarizability * -1
pol8 = ap_nn8.ref_elec_electric_polarizability * -1
pol9 = ap_nn9.ref_elec_electric_polarizability * -1
pol10 = ap_nn10.ref_elec_electric_polarizability * -1
pol11 = ap_nn11.ref_elec_electric_polarizability * -1
pol12 = ap_nn12.ref_elec_electric_polarizability * -1

In [18]:
numerical_2nd_alchemical_derivative = np.zeros((6, 6))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)
numerical_2nd_alchemical_derivative[2] = (an5 - an6) / (2 * fc_param)
numerical_2nd_alchemical_derivative[3] = (an7 - an8) / (2 * fc_param)
numerical_2nd_alchemical_derivative[4] = (an9 - an10) / (2 * fc_param)
numerical_2nd_alchemical_derivative[5] = (an11 - an12) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [19]:
numerical_3rd_alchemical_derivative = np.zeros((6, 6, 6))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)
numerical_3rd_alchemical_derivative[2, :, :] = (ann5 - ann6) / (2 * fc_param)
numerical_3rd_alchemical_derivative[3, :, :] = (ann7 - ann8) / (2 * fc_param)
numerical_3rd_alchemical_derivative[4, :, :] = (ann9 - ann10) / (2 * fc_param)
numerical_3rd_alchemical_derivative[5, :, :] = (ann11 - ann12) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [20]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [21]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [22]:
numer_elec_electric_dipole_gradient = np.zeros((6, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)
numer_elec_electric_dipole_gradient[2] = (dipole5 - dipole6) / (2 * fc_param)
numer_elec_electric_dipole_gradient[3] = (dipole7 - dipole8) / (2 * fc_param)
numer_elec_electric_dipole_gradient[4] = (dipole9 - dipole10) / (2 * fc_param)
numer_elec_electric_dipole_gradient[5] = (dipole11 - dipole12) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [23]:
numer_elec_electric_pol_gradient = np.zeros((6, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)
numer_elec_electric_pol_gradient[2] = (pol5 - pol6) / (2 * fc_param)
numer_elec_electric_pol_gradient[3] = (pol7 - pol8) / (2 * fc_param)
numer_elec_electric_pol_gradient[4] = (pol9 - pol10) / (2 * fc_param)
numer_elec_electric_pol_gradient[5] = (pol11 - pol12) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [24]:
print(analytical_1st_alchemical_derivative)

[-14.69556852  -1.13002724  -1.13002724  -1.11864327 -22.3496097
  -1.01764698]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [25]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 1.56663894e-07  1.21511862e-07  1.21391681e-07  9.54229513e-08
  1.87632615e-07 -1.86367585e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [26]:
print(analytical_2nd_alchemical_derivative)

[[-2.98251879  0.40535361  0.40535361  0.40313263  0.30192006  0.27684405]
 [ 0.40535361 -1.08469606  0.32594434  0.33120622  0.28192602  0.26925656]
 [ 0.40535361  0.32594434 -1.08469606  0.33120622  0.28192602  0.26925656]
 [ 0.40313263  0.33120622  0.33120622 -1.08392396  0.27796653  0.27601045]
 [ 0.30192006  0.28192602  0.28192602  0.27796653 -3.60315557  0.44664312]
 [ 0.27684405  0.26925656  0.26925656  0.27601045  0.44664312 -0.95727943]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [27]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 7.39168504e-07 -7.12509994e-08 -7.12564371e-08  1.30480171e-07
  -5.55031282e-07 -2.79688045e-07]
 [ 8.56311472e-07  5.68443107e-07 -6.27556186e-07 -1.01019155e-08
  -1.44452946e-07 -2.75699670e-07]
 [ 8.56244306e-07 -6.27589048e-07  5.68441084e-07 -1.00816385e-08
  -1.44478340e-07 -2.75706552e-07]
 [ 7.58220697e-07 -1.21606943e-07 -1.21603097e-07  2.49678447e-08
  -1.40015140e-07  7.65778770e-08]
 [ 4.73149814e-07 -2.83418254e-08 -2.83419057e-08 -6.05831234e-08
  -6.46568421e-08 -8.10436594e-08]
 [ 4.68552980e-07 -3.69957160e-07 -3.69952052e-07 -7.75124831e-07
   1.59149539e-07 -4.98639268e-08]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [28]:
print(analytical_3rd_alchemical_derivative)

[[[-0.26923386  0.08290763  0.08290763  0.08173221  0.04068338
    0.03735554]
  [ 0.08290763  0.11297092  0.075523    0.07688749  0.04816134
    0.04270145]
  [ 0.08290763  0.075523    0.11297092  0.07688749  0.04816134
    0.04270145]
  [ 0.08173221  0.07688749  0.07688749  0.10926678  0.04615448
    0.0386318 ]
  [ 0.04068338  0.04816134  0.04816134  0.04615448  0.02184993
    0.01430208]
  [ 0.03735554  0.04270145  0.04270145  0.0386318   0.01430208
    0.02408054]]

 [[ 0.08290763  0.11297092  0.075523    0.07688749  0.04816134
    0.04270145]
  [ 0.11297092 -0.85251327  0.07055889  0.07111204  0.05262468
    0.04236894]
  [ 0.075523    0.07055889  0.07055889  0.03669421  0.02811253
    0.02466812]
  [ 0.07688749  0.07111204  0.03669421  0.07004581  0.02764435
    0.02278396]
  [ 0.04816134  0.05262468  0.02811253  0.02764435  0.02931532
    0.02123612]
  [ 0.04270145  0.04236894  0.02466812  0.02278396  0.02123612
    0.03413623]]

 [[ 0.08290763  0.075523    0.11297092  0.076887

$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [29]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 3.07489434e-07  1.58695652e-07  1.58695056e-07 -2.25697773e-08
   -2.24282964e-08 -2.09427951e-07]
  [ 1.58695652e-07  1.68140892e-07  3.22039653e-08 -3.93982734e-08
    8.12488552e-08 -2.35885891e-08]
  [ 1.58695056e-07  3.22039653e-08  1.68136319e-07 -3.93989976e-08
    8.12477015e-08 -2.35890434e-08]
  [-2.25697773e-08 -3.93982734e-08 -3.93989976e-08 -9.63549260e-08
   -4.03270932e-08 -3.90628038e-08]
  [-2.24282964e-08  8.12488552e-08  8.12477015e-08 -4.03270932e-08
   -3.15027476e-07 -1.11445331e-07]
  [-2.09427951e-07 -2.35885891e-08 -2.35890434e-08 -3.90628039e-08
   -1.11445331e-07 -1.78219380e-07]]

 [[ 1.93439665e-07  1.03075657e-06 -9.00710780e-07 -6.27481233e-08
   -1.91406749e-08 -1.45173886e-07]
  [ 1.03075657e-06  1.03910080e-06  3.87153966e-08  2.02524222e-07
    1.70170832e-07  1.58467425e-07]
  [-9.00710780e-07  3.87153967e-08 -9.65254684e-07 -2.87952419e-07
   -2.08807884e-07 -2.72505377e-07]
  [-6.27481233e-08  2.02524222e-07 -2.87952419e-07 -1.18232599e-07
   -

$\partial^2E/\partial Z_i\partial R_j$

In [30]:
print(efficient_analytical_alchemical_force)

[[[ 0.0213926   0.00618461 -0.02157777]
  [ 0.02244613 -0.06699425  0.04102775]
  [-0.06730211 -0.02812237 -0.03680888]
  [-0.0218055   0.05715793  0.05368733]
  [ 0.04355232  0.04025592 -0.03011333]
  [ 0.00171656 -0.00848184 -0.00621511]]

 [[-0.01961834 -0.01817134  0.03089087]
  [ 0.00802809 -0.05398191  0.03035829]
  [-0.0118829   0.01221784 -0.01000215]
  [-0.00813519  0.01964461 -0.00661723]
  [ 0.03683063  0.04513042 -0.03934857]
  [-0.00522229 -0.00483963 -0.00528121]]

 [[-0.03512848 -0.01145356  0.01743927]
  [ 0.01476915  0.00067427  0.01311257]
  [-0.05150272 -0.02819784 -0.02127145]
  [ 0.01494252  0.00964916  0.01339756]
  [ 0.05419609  0.03760907 -0.02428789]
  [ 0.00272344 -0.00828109  0.00160994]]

 [[-0.0242235  -0.0092752   0.02329843]
  [ 0.0061057  -0.0172533  -0.01150252]
  [ 0.00239123 -0.01564448 -0.01472399]
  [-0.01878045  0.04379734  0.04352703]
  [ 0.04367619  0.00907755 -0.04582674]
  [-0.00916917 -0.01070191  0.00522779]]

 [[ 0.03916132  0.02783394 -0.03

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [31]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[ 2.16335573e-08  9.38028486e-08  2.19115384e-08]
  [-6.49215520e-08 -1.12521838e-07  7.82892342e-08]
  [-1.18245021e-07 -8.94337570e-08  3.20434732e-08]
  [ 8.62273294e-08  8.46701161e-08 -5.71361158e-08]
  [-6.59086796e-08  1.11453090e-07  1.31690226e-07]
  [ 1.41234183e-07 -8.79771010e-08 -2.06782989e-07]]

 [[ 3.88827541e-08  9.58471629e-08 -1.85377959e-08]
  [ 4.01901869e-08 -8.01386735e-07  7.66389150e-07]
  [ 7.75967058e-07  1.89183857e-07  1.03035841e-07]
  [ 3.59133600e-08 -1.65451866e-08  1.22289787e-08]
  [-1.56849597e-06  8.93231844e-07 -1.04038807e-06]
  [ 6.77547054e-07 -3.60347921e-07  1.77272919e-07]]

 [[ 5.81743467e-08  8.74975695e-08 -1.82299653e-09]
  [-3.17455423e-08  5.39014998e-07 -5.97462785e-07]
  [-1.04443102e-06 -3.31604488e-07 -1.74283561e-07]
  [-1.94409641e-08  7.42580560e-09 -3.57825977e-08]
  [ 1.37791078e-06 -3.82931066e-07  1.51495673e-06]
  [-3.40475501e-07  8.05858047e-08 -7.05635487e-07]]

 [[-9.95211429e-08 -1.15177568e-07  5.72259700e-08]
  [ 3.

$\partial^2E/\partial Z_i\partial R_j$

In [32]:
# print(inefficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [33]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,efficient}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,inefficient}$

In [34]:
# print(efficient_analytical_alchemical_force - inefficient_analytical_alchemical_force)

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [35]:
print(elec_electric_dipole_gradient_f)

[[-0.04234378 -0.0730948   0.01231999]
 [ 0.35743473 -1.34593223  0.78518923]
 [-1.31427724 -0.62187791 -0.66464914]
 [-0.52606071  0.93199921  1.07200918]
 [ 0.63343031  0.21466197 -0.62316334]
 [ 0.87931238  1.49323811 -0.26814819]]


In [36]:
print(elec_electric_dipole_gradient_z)

[[-0.04234378 -0.07309481  0.01231999]
 [ 0.35743474 -1.34593223  0.78518923]
 [-1.31427725 -0.6218779  -0.66464914]
 [-0.52606067  0.93199923  1.07200915]
 [ 0.63343035  0.21466199 -0.62316336]
 [ 0.87931236  1.49323809 -0.26814818]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [37]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[ 2.56996882e-07  1.49369239e-06  4.49635550e-07]
 [-8.59892862e-06  4.45216920e-06 -5.59905608e-06]
 [ 7.26195991e-06 -2.41750615e-06  8.15672513e-06]
 [-1.14467844e-06  1.40442514e-06  2.02122141e-06]
 [ 3.99526459e-07 -1.48730583e-07 -5.34945715e-07]
 [-1.90050576e-06  1.61041221e-06  2.99559783e-06]]


In [38]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[ 2.57646997e-07  1.49010729e-06  4.47095526e-07]
 [-8.58898181e-06  4.45588602e-06 -5.59127902e-06]
 [ 7.25635654e-06 -2.40705421e-06  8.15101585e-06]
 [-1.10348116e-06  1.42533292e-06  1.98416100e-06]
 [ 4.30373602e-07 -1.29721193e-07 -5.61020142e-07]
 [-1.91662796e-06  1.58675212e-06  3.00237134e-06]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [39]:
print(elec_electric_pol_gradient)

[[[ 3.50226186 -0.93583965  1.71807968]
  [-0.93583965  4.94567969  0.85334074]
  [ 1.71807968  0.85334074  3.8428043 ]]

 [[ 4.98419818 -1.27123916  1.59389107]
  [-1.27123916  7.29676063 -0.45408766]
  [ 1.59389107 -0.45408766  5.28389194]]

 [[ 6.17145584  0.1008236   2.23787447]
  [ 0.1008236   5.88549838  0.90292598]
  [ 2.23787447  0.90292598  5.5078966 ]]

 [[ 4.80018563 -0.79834151  0.54869179]
  [-0.79834151  6.86367628  1.54069547]
  [ 0.54869179  1.54069547  5.75860131]]

 [[ 5.77566248  0.53783799 -0.57259623]
  [ 0.53783799  5.28724848 -0.49972143]
  [-0.57259623 -0.49972143  5.45677334]]

 [[ 4.29132533  0.28202384 -0.58605928]
  [ 0.28202384  3.63154146 -0.33984395]
  [-0.58605928 -0.33984395  4.1669253 ]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [40]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[ 2.88996723e-06  1.16818659e-05 -8.98774850e-06]
  [ 1.16818659e-05 -7.71718855e-06 -1.23470708e-05]
  [-8.98774850e-06 -1.23470708e-05 -5.76770647e-06]]

 [[-3.72034337e-05 -5.43562424e-06 -1.77798313e-05]
  [-5.43562424e-06  4.06140687e-06 -4.40867937e-05]
  [-1.77798313e-05 -4.40867937e-05  8.78168825e-06]]

 [[ 4.21699239e-05  2.94757542e-05 -3.10093454e-06]
  [ 2.94757542e-05 -4.10700379e-05  9.64856331e-06]
  [-3.10093454e-06  9.64856331e-06 -2.54592185e-05]]

 [[-4.08984813e-06  1.66180431e-05 -8.10801414e-06]
  [ 1.66180431e-05 -2.50874178e-05 -2.25417326e-05]
  [-8.10801414e-06 -2.25417326e-05 -2.02278625e-05]]

 [[ 1.81645885e-06  1.30896167e-05 -1.86575856e-05]
  [ 1.30896167e-05  1.33663140e-05  1.58711485e-06]
  [-1.86575856e-05  1.58711485e-06  2.27132878e-06]]

 [[ 2.62561810e-05  2.42552552e-05 -3.98064202e-05]
  [ 2.42552552e-05  1.51360921e-05 -1.10330882e-05]
  [-3.98064202e-05 -1.10330882e-05  2.16156251e-05]]]
